In [ ]:
# Import libraries and modules
import time
from tqdm import tqdm

import math

import numpy as np

import scipy as sp

import pandas as pd

import theano

from keras import backend as K
K.set_image_dim_ordering('th')

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.utils import np_utils
from keras.utils.np_utils import to_categorical

from keras.layers import LSTM

from sklearn.preprocessing import MinMaxScaler

import os
import gc

raster = 200
crime_type = "E05"
date_from = "2013-06-16"
date_to = "2017-03-08"

days = 21
dist = 5


# insert main dir
main_dir = ""

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.font_manager as font_manager
import matplotlib.gridspec as gridspec
import matplotlib.colors as col

from sklearn.metrics import roc_curve, auc
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import confusion_matrix
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

import copy

from numpy.ma import masked_array

import seaborn as sns

startcolor = '#fff2e5'
midcolor = '#f2a285'
endcolor = '#c40d21'    
cmap1 = col.LinearSegmentedColormap.from_list('own1',[startcolor,midcolor,endcolor])
plt.cm.register_cmap(name = 'wrong', cmap=cmap1)

startcolor = '#f3ffe5' 
midcolor = '#b4f185'   
endcolor = 'darkgreen'   
cmap2 = col.LinearSegmentedColormap.from_list('own2',[startcolor,midcolor,endcolor])
plt.cm.register_cmap(name = 'correct', cmap=cmap2)

classes=[0,1]

def find_max_mcc_threshold(true_values, predictions, thresholds):
    max_mcc = -1
    max_threshold = thresholds[0]
    max_predictions = predictions[:]
    stop = 0
    for i in range(1,thresholds.shape[0]):
        tmp_threshold = thresholds[i]
        tmp_predictions = copy.deepcopy(predictions)
        tmp_predictions[tmp_predictions >= tmp_threshold] = int(1)
        tmp_predictions[tmp_predictions < 1] = int(0)
        tmp_mcc = matthews_corrcoef(true_values, tmp_predictions)
        if tmp_mcc > max_mcc:
            max_mcc = tmp_mcc
            max_threshold = tmp_threshold
            max_predictions = copy.deepcopy(tmp_predictions)
            stop = 0
        else:
            stop += 1
        if stop == 3000:
            break;
    return max_mcc, max_threshold, max_predictions.astype(int)

    
def print_save_single_roc_threshold(true_values, predictions, title, fig_name):
    fpr, tpr, thresholds = roc_curve(true_values, predictions)
    prediction_auc = auc(fpr, tpr)
    
    mcc_max, threshold, predictions = find_max_mcc_threshold(true_values, predictions, thresholds)
    acc = accuracy_score(true_values, predictions)
    f1 = f1_score(true_values, predictions)
    th_index = thresholds.tolist().index(threshold)

    title_font = { 'size':'17', 'color':'black', 'weight':'normal', 'verticalalignment':'bottom'}
    axis_font = {'size':'15'}
    cb_font = {'size':'15', 'horizontalalignment':'left'}
    font_path = "/usr/share/fonts/truetype/msttcorefonts/Arial.ttf"
    font_prop = font_manager.FontProperties(size=15)
    
    
    fig = plt.figure(figsize=(15,7))
    gs1 = gridspec.GridSpec(1, 2)
    ax_list = [fig.add_subplot(ss) for ss in gs1]
    ax1 =  ax_list[0]
    ax2 =  ax_list[1]
    lw = 2

    ax1.scatter(fpr[th_index], tpr[th_index], s=75, c="red", label='Optimal threshold \n(cutoff = %0.4f)' % (threshold))
    ax1.plot(fpr, tpr, color="black", lw=lw, label='ROC curve (area = %0.4f)' % prediction_auc)
    ax1.plot([0, 1], [0, 1], color='gray', lw=lw, linestyle='--')
    ax1.set_xlim([0.0, 1.0])
    ax1.set_ylim([0.0, 1.05])
    ax1.set_xlabel('False Positive Rate', **axis_font)
    ax1.set_ylabel('True Positive Rate', **axis_font)
    ax1.set_title("Roc analysis",  **title_font)
    ax1.legend(loc="lower right", prop= font_prop)
    
    cm = confusion_matrix(true_values, predictions)
    cm_n = cm / cm.sum(axis=1)[:, np.newaxis]
    cm_f = [[1,0],[1,0]]
    
    mask1 = [[0,  1], [1, 0]]
    mask2 = [[1,  0], [0, 1]]
    cm1 = masked_array(cm_n,mask1)
    cm2 = masked_array(cm_n,mask2)
    cm1f = masked_array(cm_f,mask1)
    cm2f = masked_array(cm_f,mask2)
    
    cmap1=plt.cm.get_cmap("correct")
    cmap2=plt.cm.get_cmap("wrong")
    
    p2f = ax2.imshow(cm2f,interpolation='nearest',cmap=cmap2)
    p1f = ax2.imshow(cm1f,interpolation='nearest',cmap=cmap1)
   
    p2 = ax2.imshow(cm2,interpolation='nearest',cmap=cmap2)
    p1 = ax2.imshow(cm1,interpolation='nearest',cmap=cmap1)
    
    cb2 = plt.colorbar(p2,shrink=0.5)
    cb2.set_clim(0, 1)
    cb2.remove()
    
    cb1 = plt.colorbar(p1,shrink=0.5)
    cb1.set_clim(0, 1)
    cb1.remove()
    
    cb2 = plt.colorbar(p2f,shrink=0.5)
    cb2.set_clim(0, cm1.sum())
    cb2.ax.get_xaxis().labelpad = 10
    cb2.ax.set_xlabel('False', **cb_font)
    cb2.ax.tick_params(labelsize=15)

    cb1 = plt.colorbar(p1f,shrink=0.5)
    cb1.set_clim(0, cm1.sum())
    cb1.ax.get_xaxis().labelpad = 10
    cb1.ax.set_xlabel('True', **cb_font)
    cb1.ax.tick_params(labelsize=15)

    ax2.grid(False)

    ax2.set_title("Confusion matrix\n\nACC: %0.3f   F1: %0.3f   MCC: %0.3f" % (acc, f1, mcc_max), **title_font)

    tick_marks = np.arange(len(classes))
    ax2.set_xticks(tick_marks)
    ax2.set_yticks(tick_marks)
    ax2.set_xticklabels(classes)
    ax2.set_yticklabels(classes)
    
    thresh = 0.6

    ax2.text(0, 0, "%d\n(%0.2f)" % (cm[0,0], cm_n[0,0]), horizontalalignment="center", color="white" if cm_n[0, 0] > thresh else "black", fontproperties=font_prop)
    ax2.text(1, 1, "%d\n(%0.2f)" % (cm[1,1], cm_n[1,1]), horizontalalignment="center", color="white" if cm_n[1, 1] > thresh else "black", fontproperties=font_prop)
    ax2.text(1, 0, "%d\n(%0.2f)" % (cm[0,1], cm_n[0,1]), horizontalalignment="center", color="white" if cm_n[0, 1] > thresh else "black", fontproperties=font_prop)
    ax2.text(0, 1, "%d\n(%0.2f)" % (cm[1,0], cm_n[1,0]), horizontalalignment="center", color="white" if cm_n[1, 0] > thresh else "black", fontproperties=font_prop)

    ax2.set_ylabel('True label', **axis_font)
    ax2.set_xlabel('Predicted label', **axis_font)
    
    for label in (ax2.get_xticklabels() + ax2.get_yticklabels() + ax1.get_xticklabels() + ax1.get_yticklabels()):
        label.set_fontsize(13)
    
    plt.suptitle(title, fontsize=20)   
    
    plt.tight_layout(pad=1, w_pad=1, h_pad=1)
    plt.savefig("%s/output/python/keras/images/%s.png" % (main_dir, fig_name))
    plt.show()
    return "%0.5f,%0.5f,%0.5f,%0.5f,%0.5f,%d,%d,%d,%d,%0.5f,%0.5f,%0.5f,%0.5f" % (prediction_auc,acc,f1,mcc_max,threshold,cm[0,0],cm[1,1],cm[0,1],cm[1,0], cm_n[0,0],cm_n[1,1],cm_n[0,1],cm_n[1,0])
    


In [ ]:
dir_data_tm = "%s/data/%s/tmdata_%d_%d_%s_%s_%s_s_select.csv" % (main_dir, crime_type, days, raster, crime_type, date_from, date_to)
data_tm = pd.read_csv(dir_data_tm)

data_tm = data_tm[data_tm.date > '2013-07-06']

data_tm.sort_values('date', inplace=True)
print(data_tm.shape)
data_tm.head()

In [ ]:
# Split data

train_days_percent = 0.7
validation_days_percent = 0.2

ids_count = np.array((data_tm.drop_duplicates("id", inplace = False)).iloc[:,0]).shape[0]
rows = data_tm.shape[0]

total_days = rows/ids_count

train_rows = int(ids_count * total_days * train_days_percent)
validation_rows = int(ids_count * total_days * validation_days_percent)
test_rows = rows - train_rows - validation_rows
print("Train rows: %d, validation rows: %d, test rows: %d, all: %d < %d" % (train_rows, validation_rows, test_rows, (train_rows+validation_rows+test_rows), rows))


X_train = (data_tm.iloc[0:train_rows,6:]).values
X_validation = (data_tm.iloc[(train_rows):(train_rows+validation_rows),6:]).values
X_test = (data_tm.iloc[(train_rows+validation_rows):,6:]).values

y_train = data_tm.iloc[0:train_rows, 5].values
y_validation = data_tm.iloc[(train_rows):(train_rows+validation_rows),5].values
y_test = data_tm.iloc[(train_rows+validation_rows):,5].values

n_train = y_train.shape[0]
n_validation = y_validation.shape[0]
n_test = y_test.shape[0]

print(n_train, n_validation, n_test, (n_train + n_validation + n_test), rows)
print((data_tm.iloc[0:train_rows,:]).date.min(), (data_tm.iloc[(train_rows):(train_rows+validation_rows),:]).date.min(), (data_tm.iloc[(train_rows+validation_rows):,:]).date.min())
print(y_test[y_test > 0].sum())


max_y = int(data_tm["crimecount"].max())

normalize_y = False

if normalize_y:
    y_train /= max_y
    y_validation /= max_y
    y_test /= max_y
    
normalize_x = True

if normalize_x:
    # normalize from [0, max] to [0, 1] so max + 1 clasess
    scaler = MinMaxScaler(feature_range=(0, 1))
    X_train = scaler.fit_transform(X_train)
    X_validation = scaler.fit_transform(X_validation)
    X_test = scaler.fit_transform(X_test)

binary = True

if binary:
    np.clip(y_train, 0, 1, out=y_train)
    np.clip(y_validation, 0, 1, out=y_validation)
    np.clip(y_test, 0, 1, out=y_test)
    n_classes = 2
    
    max_y = 1
    data_tm.crimecount[data_tm["crimecount"] > 0] = 1
    hist, bin_edges = np.histogram(data_tm["crimecount"])
    plt.bar(bin_edges[:-1], hist, width = 1)
    plt.xlim(min(bin_edges), max(bin_edges))
    plt.show()
    print(hist)
else:
    n_classes = max_y + 1
    hist, bin_edges = np.histogram(data[y])
    plt.bar(bin_edges[:-1], hist, width = 1)
    plt.xlim(min(bin_edges), max(bin_edges))
    plt.show() 
    print(hist)


In [ ]:
# Reshape - timestep
days = 21
X_train = X_train.reshape(n_train, days, 1).astype('float16')
X_validation = X_validation.reshape(n_validation, days, 1).astype('float16')
X_test = X_test.reshape(n_test, days, 1).astype('float16')


In [ ]:
result_file_dir = "%s/output/python/model_results.csv" % main_dir
data_id = "tm_data"

batch_size = int(X_train.shape[0]/(int(X_train.shape[0]/1298)*0.7))

epochs = 1

def train_test_model_save_results_rnn_sf(model, model_id):
    model.fit(X_train,
          y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(X_validation, y_validation))
    
    save_dir_model = "%s/output/python/keras/models/%s_%d_%s_%s_%s_d_%d_select_%s.h5" % (main_dir, model_id, raster,crime_type, date_from, date_to, dist, data_id)
    model.save(save_dir_model)

    prediction = model.predict(X_test)

    result_line = print_save_single_roc_threshold(y_test[:,1], prediction[:,1], "", "roc_%s_%s_th" % (data_id, model_id))
    params = str(model.to_json())
    with open(result_file_dir, 'a') as file:
        file.write("%s,%s,%s,%s\n" % (model_id, data_id, result_line, params))
        
def train_test_model_save_results_rnn(model, model_id):
    model.fit(X_train,
          y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(X_validation, y_validation))
    
    save_dir_model = "%s/output/python/keras/models/%s_%d_%s_%s_%s_d_%d_select_%s.h5" % (main_dir, model_id, raster,crime_type, date_from, date_to, dist, data_id)
    model.save(save_dir_model)

    prediction = model.predict(X_test)

    result_line = print_save_single_roc_threshold(y_test, prediction, "", "roc_%s_%s_th" % (data_id, model_id))
    params = str(model.to_json())
    with open(result_file_dir, 'a') as file:
        file.write("%s,%s,%s,%s\n" % (model_id, data_id, result_line, params))

In [ ]:
# create and the LSTM network timestep, classification 1
model = Sequential()
model.add(LSTM(50, input_shape=(days, 1)))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['binary_accuracy'])

train_test_model_save_results_rnn(model, 'rnn_1')
print("Done.")

In [ ]:
# create and the LSTM network timestep, classification 2
model = Sequential()
model.add(LSTM(100, input_shape=(days, 1)))
model.add(Dropout(0.2))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['binary_accuracy'])

train_test_model_save_results_rnn(model, 'rnn_2')
print("Done.")

In [ ]:
# create and the LSTM network timestep, classification 3
model = Sequential()
model.add(LSTM(121, input_shape=(days, 1)))
model.add(Dropout(.2))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['binary_accuracy'])

train_test_model_save_results_rnn(model, 'rnn_3')
print("Done.")

In [ ]:
# create and the LSTM network timestep, classification 4
model = Sequential()

model.add(LSTM(21, input_shape=(days, 1), return_sequences=True))
model.add(LSTM(21, input_shape=(days, 1), return_sequences=True))
model.add(LSTM(21, input_shape=(days, 1), return_sequences=True))
model.add(LSTM(21, input_shape=(days, 1)))
model.add(Dropout(0.2))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['binary_accuracy'])

train_test_model_save_results_rnn(model, 'rnn_4')
print("Done.")

In [ ]:
# create and the LSTM network timestep, classification 5
model = Sequential()

model.add(LSTM(21, input_shape=(days, 1), return_sequences=True))
model.add(LSTM(21, input_shape=(days, 1), return_sequences=True))
model.add(LSTM(21, input_shape=(days, 1), return_sequences=True))
model.add(LSTM(21, input_shape=(days, 1), return_sequences=True))
model.add(LSTM(21, input_shape=(days, 1), return_sequences=True))
model.add(LSTM(21, input_shape=(days, 1), return_sequences=True))
model.add(LSTM(21, input_shape=(days, 1), return_sequences=True))
model.add(LSTM(21, input_shape=(days, 1)))
model.add(Dropout(0.2))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['binary_accuracy'])

train_test_model_save_results_rnn(model, 'rnn_5')
print("Done.")

In [ ]:
# create and the LSTM network timestep, classification, 6
model = Sequential()

model.add(LSTM(32, input_shape=(days,1), return_sequences=True))
model.add(LSTM(32, input_shape=(days,1)))

model.add(Dropout(.2))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['binary_accuracy'])
model
    
train_test_model_save_results_rnn(model, 'rnn_6')

print("Done.")

In [ ]:
# create and the LSTM network timestep, classification 7
model = Sequential()

model.add(LSTM(32, input_shape=(days,1), return_sequences=True))
model.add(LSTM(32, input_shape=(days,1), return_sequences=True))
model.add(LSTM(32, input_shape=(days,1), return_sequences=True))
model.add(LSTM(32, input_shape=(days,1)))

model.add(Dropout(.2))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['binary_accuracy'])
model
    
train_test_model_save_results_rnn(model, 'rnn_7')

print("Done.")

In [ ]:
# create and the LSTM network timestep, classification 8
model = Sequential()

model.add(LSTM(32, input_shape=(days,1), return_sequences=True))
model.add(LSTM(32, input_shape=(days,1), return_sequences=True))
model.add(LSTM(32, input_shape=(days,1), return_sequences=True))
model.add(LSTM(32, input_shape=(days,1), return_sequences=True))
model.add(LSTM(32, input_shape=(days,1), return_sequences=True))
model.add(LSTM(32, input_shape=(days,1), return_sequences=True))
model.add(LSTM(32, input_shape=(days,1), return_sequences=True))
model.add(LSTM(32, input_shape=(days,1)))

model.add(Dropout(.2))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['binary_accuracy'])
model
    
train_test_model_save_results_rnn(model, 'rnn_8')

print("Done.")

In [ ]:
# create and the LSTM network timestep, classification 9
model = Sequential()
model.add(LSTM(8, input_shape=(days, 1), return_sequences=True))
model.add(LSTM(8, input_shape=(days, 1), return_sequences=True))
model.add(LSTM(8, input_shape=(days, 1), return_sequences=True))
model.add(LSTM(8, input_shape=(days, 1), return_sequences=True))
model.add(LSTM(8, input_shape=(days, 1), return_sequences=True))
model.add(LSTM(8, input_shape=(days, 1), return_sequences=True))
model.add(LSTM(8, input_shape=(days, 1), return_sequences=True))
model.add(LSTM(8, input_shape=(days, 1)))
model.add(Dropout(.2))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['binary_accuracy'])

train_test_model_save_results_rnn(model, 'rnn_9')
print("Done.")

In [ ]:
# create and the LSTM network timestep, classification 10
model = Sequential()
model.add(LSTM(254, input_shape=(days, 1)))
model.add(Dropout(.2))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['binary_accuracy'])

train_test_model_save_results_rnn(model, 'rnn_10')
print("Done.")

In [ ]:
# create and the LSTM network timestep, classification 11
model = Sequential()
model.add(LSTM(100, input_shape=(days, 1), return_sequences=True))
model.add(LSTM(100, input_shape=(days, 1)))
model.add(Dropout(.2))
model.add(Dense(100))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['binary_accuracy'])

train_test_model_save_results_rnn(model, 'rnn_11')
print("Done.")

In [ ]:
# create and the LSTM network timestep, classification 12
model = Sequential()
model.add(LSTM(100, input_shape=(days, 1),return_sequences=True))
model.add(LSTM(100, input_shape=(days, 1)))
model.add(Dropout(.2))
model.add(Dense(100))
model.add(Dense(100))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['binary_accuracy'])

train_test_model_save_results_rnn(model, 'rnn_12')
print("Done.")

## Plot result

In [ ]:
#from keras.models import load_model
#load_dir_model = "%s/output/python/rnn_model_%d_%s_%s_%s_d_%d.h5" % (main_dir, raster,crime_type, date_from, date_to, dist)
#model = load_model(load_dir_model)

dir_data_tm = "%s/data/%s/tmdata_%d_%d_%s_%s_%s.csv" % (main_dir, crime_type, days, raster, crime_type, date_from, date_to)
data_tm = pd.read_csv(dir_data_tm)
print(data_tm.shape)
data_tm.head()

In [ ]:
from keras.models import load_model
load_dir_model = "%s/output/python/rnn_model_%d_%s_%s_%s_d_%d_select3.h5" % (main_dir, raster,crime_type, date_from, date_to, dist)
model = load_model(load_dir_model)

In [ ]:
ids = np.array((data_tm.drop_duplicates("id", inplace = False, )).iloc[:,0])
print(ids.shape)

for i in range(2005, 2015):
    rec_id = ids[i]
    rectangle_history = data_tm.loc[(data_tm['id']==rec_id)]
    data_X = np.array(rectangle_history.iloc[:,6:])
    data_y = np.array(rectangle_history.iloc[:,5])

    data_X = data_X.reshape((data_X.shape[0], data_X.shape[1], 1)).astype('float32')
    data_y = data_y.astype('float32')
    
    data_X /= max_y
    data_y /= max_y

    prediction = model.predict(data_X)
    print(prediction.mean())
    #prediction_t = prediction.reshape(969)
    #prediction_t[prediction_t != -0.00084887573] = 1
    #prediction_t[prediction_t == -0.00084887573] = 0
    #data_y[data_y > 0] = 1
    
    plt.clf
    plt.figure(figsize=(20,10))
    plt.plot(prediction)
    plt.plot(data_y)
    
    plt.show()